### Model insight and analysis

**Previous model:** We assum that, each word in the input sentence have a weight affect to the label. To this end, we need to learn these hidden weights. *We designed a Embedding layer (random init) to convert word to vector. Then `sum`  embedding vectors of all words in a input sentence, then transform document vector to the label values (images)*

![model_arc](../../img/dl_tutorial-Trang-2.drawio.png)

**Questions?** 
- If the order of words in the sentence change, the performance of this model will change or not? Why?
- what difference thing model learn in embedding layer and the Linear output layer ?  
- what did model learn? or which words strongly affected to the label ? For example, given sentence "im updating my because i feel shitty" => label is "sadness" , how we know that model learned the correct thing that shitty is the sad emotion? 
- Can we manually create a embedding vector for a new word (e.g. "wa_ta_shi") to achieve current performance without train model again? 

In [1]:
import torch
import numpy as np 
 
from torch.utils.data import DataLoader
import torch

from transformers import BertConfig, AutoTokenizer, AutoModel

from torchtext.vocab import vocab
from collections import Counter, OrderedDict
from datasets import load_dataset
import re
import random
# =====================

def set_random_seed(seed: int):
    """set seeds for reproducibility"""
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    
set_random_seed(7)

dataset = load_dataset('emotion')
train_data = dataset['train']
all_labels = []
for sample in train_data:
    all_labels.append(sample['label'])


# count label 
num_labels = len(set(all_labels))
 

# init model 
# Load config from pretrained name or path 
pre_trained_model_name = 'roberta-base'
config = BertConfig.from_pretrained(pre_trained_model_name)  # Load pretrained bert
bert_tokenizer = AutoTokenizer.from_pretrained(pre_trained_model_name)

class BatchPreprocessor(object): 

    def __call__(self, batch):
        raw_sentences = []

        # collect all sentences
        for sample in batch:
            raw_sentences.append(sample['text'])

        # label processing 
        labels = []
        for sample in batch:
            label = sample['label']
            labels.append(int(label))

        word_ids_from_bert_tokenizer = bert_tokenizer(raw_sentences,  padding='max_length', max_length=512, truncation=True, return_tensors='pt')

        return (word_ids_from_bert_tokenizer, torch.FloatTensor(labels), raw_sentences) 




No config specified, defaulting to: emotion/split
Found cached dataset emotion (/home/phuongnm/.cache/huggingface/datasets/emotion/split/1.0.0/cca5efe2dfeb58c1d098e0f9eeb200e9927d889b5a03c67097275dfb5fe463bd)


  0%|          | 0/3 [00:00<?, ?it/s]

You are using a model of type roberta to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.


In [2]:

batch_size = 32

# dataset_example should support operator index_selection for create the data_loader object
test_loader = DataLoader(dataset['test'], batch_size=batch_size, collate_fn=BatchPreprocessor(), shuffle=True)
train_loader = DataLoader(dataset['train'], batch_size=batch_size, collate_fn=BatchPreprocessor(), shuffle=True)
valid_loader = DataLoader(dataset['validation'], batch_size=batch_size, collate_fn=BatchPreprocessor(), shuffle=True)
for e in test_loader:
    print('First epoch data:')
    print('input data\n', e[0])
    print('label data\n',e[1])
    print('padding mask data\n',e[2])
    print(e[0]['input_ids'].device)
    break  

First epoch data:
input data
 {'input_ids': tensor([[  0, 118, 619,  ...,   1,   1,   1],
        [  0, 118, 619,  ...,   1,   1,   1],
        [  0, 118,  40,  ...,   1,   1,   1],
        ...,
        [  0, 118, 802,  ...,   1,   1,   1],
        [  0, 118,  21,  ...,   1,   1,   1],
        [  0, 118, 619,  ...,   1,   1,   1]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}
label data
 tensor([0., 0., 5., 3., 1., 0., 3., 1., 3., 1., 1., 4., 4., 0., 4., 1., 0., 0.,
        1., 0., 2., 1., 0., 1., 0., 0., 0., 2., 3., 1., 2., 3.])
padding mask data
 ['i feel i might have lost the potty training train', 'i feel quite helpless in all of this so prayer is the most effective tool i have because i have no answers and there is nothing else i can offer them right now', 'i will practice meditation if i fe

In [3]:
print('train size', len(train_loader))
print('test size',  len(test_loader))
len(my_vocab)

train size 500
test size 63


NameError: name 'my_vocab' is not defined

In [4]:
from torch import nn

d_model = 768
bert_model = AutoModel.from_pretrained(pre_trained_model_name)
bert_model.cuda()
  
output_layer = nn.Linear(d_model, 1)
output_layer.cuda()
# ===================================

# loss also is supported by a library 
loss_computation = torch.nn.MSELoss()
optimizer = torch.optim.Adam(list(bert_model.parameters()) + list(output_layer.parameters()), lr = 1e-5)    # using Adam optimizer instead of SGD



Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [5]:
def forward_function(bert_inputs, output_module): 
    
    outputs = bert_model(**bert_inputs)
    h_cls = outputs[1]
    label_vectors = output_module(h_cls) 

    return label_vectors

In [6]:
def eval(data_loader):
    count_true = 0
    count_total = 0
    for batch in data_loader:

        x, y_gold, raw_sentences = batch

        x = dict([(k, v.cuda()) for k, v in x.items()]) # HARD
        y_gold = y_gold.cuda()
        
        # ============= ###### IMPORTANT ######## ===============
        # Forward pass: Compute predicted y by passing x to the model
        label_vectors = forward_function(bert_inputs=x, output_module=output_layer)
        
        predictions = torch.ceil(label_vectors.squeeze())
        # ============= ######################### ===============
        
        count_true += torch.sum((predictions==y_gold) == True)
        count_total += y_gold.shape[0]

    return count_true / count_total
print('Acc model BEFORE train = ', eval(test_loader))

Acc model BEFORE train =  tensor(0.3475, device='cuda:0')


In [7]:
len(train_loader)

500

In [8]:


MAX_EPOCHS=1
for epoch in range(MAX_EPOCHS):
    avg_loss = 0.0
    for step, batch in enumerate(train_loader):

        x, y_gold, raw_sentences = batch

        x = dict([(k, v.cuda()) for k, v in x.items()]) # HARD
        y_gold = y_gold.cuda() 
        

        # ============= ###### IMPORTANT ######## ===============
        # Forward pass: Compute predicted y by passing x to the model
        label_vectors = forward_function(bert_inputs=x, output_module=output_layer)


        # Compute and loss = average ((out_put - pred) ^ 2)
        loss = loss_computation(label_vectors.squeeze(), y_gold) 
        if step %10 == 0:
            print(f"step {step}: loss={loss.item()}")
        # ============= ######################### ===============

        # perform a backward pass (backpropagation) => to compute the gradient values in Tensor weights
        loss.backward()
        avg_loss += loss.item()

        # USE LIBRARY: 'model.parameters()' in stead of 'model.get_parameter()' is implemented by library, also return list of parameters: "weight" and "bias" 
        # Optimizer step(), this update gradient values to weights.
        optimizer.step() # instead of `param.add_(-lr * param.grad)` => update weight values
        optimizer.zero_grad() # instead of `param.grad.fill_(0)` => remove all the old gradient values in all Tensor weight
    
    avg_loss = avg_loss / len(train_loader)
    if avg_loss < 0.0001:
        print(loss)
        break
    print('epoch/batch: ', epoch, ' avg loss: ', avg_loss, "Acc=", eval(valid_loader))
    # break 


step 0: loss=3.7480270862579346
step 10: loss=3.746635675430298
step 20: loss=1.4168243408203125
step 30: loss=1.9797720909118652
step 40: loss=3.3791303634643555
step 50: loss=2.530238151550293
step 60: loss=2.7934889793395996
step 70: loss=2.0442585945129395
step 80: loss=1.4724265336990356
step 90: loss=0.8465590476989746
step 100: loss=0.8569158315658569
step 110: loss=0.9766244888305664
step 120: loss=0.989676833152771
step 130: loss=0.7003448009490967
step 140: loss=0.7205039858818054
step 150: loss=0.7197826504707336
step 160: loss=0.8840521574020386
step 170: loss=0.6590648889541626
step 180: loss=0.2490629255771637
step 190: loss=0.5959786176681519
step 200: loss=0.5967725515365601
step 210: loss=0.3765519857406616
step 220: loss=0.9442761540412903
step 230: loss=0.5287991762161255
step 240: loss=0.10454721748828888
step 250: loss=0.5070899724960327
step 260: loss=0.4888703227043152
step 270: loss=0.5518723130226135
step 280: loss=0.30440759658813477
step 290: loss=0.286159396

In [9]:
print('Acc model AFTER train = ', eval(test_loader))


Acc model AFTER train =  tensor(0.4030, device='cuda:0')
